In [1]:
import linear_model as model
import src.project1.submit.AI as ai
from experiment.评估函数.simulator import Simulator
from experiment.评估函数.greedy_ai import GreedyAI
from experiment.评估函数.greedys import *
import numpy as np

chessboard_size = 8
time_out = 5


def greedy_fun(Vars):
    PVT, PVT_max = model.get_PVT_and_max(Vars)
    return lambda a, p: model.value_of_positions(a.reversi_env.updated_chessboard(a.chessboard, a.color, p), a.color,
                                                 PVT, PVT_max)


baselines = [random_baseline, middle_action_first, map_weight_sum, map_weight_sum2]
baseline_cnt = len(baselines)
neighbour_cnt = 10
# 注意时间复杂度不要太高
落子次数 = (4 + neighbour_cnt) * (3 + neighbour_cnt) * 60
print(落子次数)

10920


In [11]:
import random


def simulated_annealing(state, T=1, log_interval=10):
    t = 0  # time step
    f = []  # a recording of values
    while T >= 1e-7:
        vars_list = model.neighbours_continuous(state, -1, 1, 0.1, neighbour_cnt)
        funcs = baselines + list(map(greedy_fun, vars_list))
        length = len(funcs)
        matches = 2/2*length*(length-1)
        scores = np.zeros(length)
        for i in range(length):
            for j in range(i + 1, length):
                for i_color in [ai.COLOR_BLACK, ai.COLOR_WHITE]:
                    agents = {i_color: GreedyAI(chessboard_size, i_color, funcs[i]),
                              -i_color: GreedyAI(chessboard_size, -i_color, funcs[j])}
                    simulator = Simulator(chessboard_size, time_out, agents)
                    winner = simulator.quick_run(no_print=True)
                    if winner == i_color:
                        scores[i] += 1
                    elif winner == -i_color:
                        scores[j] += 1
        diff = np.inf
        new_idx = -1
        new_value = np.nan
        while not (diff > 0 or np.exp(diff / T) > np.random.uniform()):
            new_idx = random.randint(1, neighbour_cnt)
            new_value = scores[new_idx]
            diff = new_value - scores[0]

        state = vars_list[new_idx]
        f.append(new_value)
        # update time and temperature
        if t % log_interval == 0:
            print(f"step {t}: T={T}, current_value={scores[new_idx]/matches}")
        t += 1
        T = 0.999 ** t
    print(f"step {t}: T={T}, current_value={scores[new_idx]/matches}")
    return state, f

In [ ]:
import matplotlib.pyplot as plt

# define your own schedule and halt condition
solution, record = simulated_annealing(
    np.array([0.58294678, 0.32422638, 0.30072021, 0.04949951, 0.74310493,
               0.26721382, 0.05133057, 0.72602844, -0.12939453, -0.73083496])
)
solution

step 0: T=1, current_value=0.06190476190476191


In [ ]:
# visualize the curve
plt.plot(record)
plt.xlabel("time step")
plt.ylabel("value")

## Deprecated: 

In [ ]:
def imerge(a, b):
    i1 = iter(a)
    i2 = iter(b)
    while True:
        try:
            yield i1.next()
            yield i2.next()
        except StopIteration:
            return


for i in imerge(range(10), range(8)):
    print(i)

In [ ]:
def schedule(t):
    T = 1000
    while True:
        yield T
        T *= 0.98


def simulated_annealing(InitialVars, rounds):
    current = InitialVars
    for t in range(rounds):
        T = schedule(t)
        if T <= 10:
            return current
        for var in imerge(current, model.neighbours(current))
